Main
Michael de Jong

In [21]:
# imports
import os
import json
from elasticsearch import Elasticsearch
import numpy as np
import pandas as pd
import nbconvert
import nbformat
import zipfile
import csv

In [51]:
# load local python file(s)
%run evaluationmetrics.py

In [52]:
# 1 = relevant, 0 = not relevant
l = [1,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1]
c = 8
print(precision(l))
print(recall(l,c))
print(f1(l,c))
print(avg_precision(l,c))

0.3
0.75
0.4285714285714285
0.262359022556391


In [22]:
# connect to local elastic search host
HOST = 'http://localhost:9200/'
es = Elasticsearch(hosts=[HOST])

# global vars
INDEX="notebookindex"
TYPE= "notebook"
COLUMNS = ['nb_id', 'html_url', 'name', 'language', 'markdown', 'comments']

In [23]:
# load the csv with all notebook information
df_nb = pd.read_csv('notebooks.csv')
df_nb = df_nb.drop(['max_filesize','min_filesize', 'path', 'query_page', 'repo_id'], axis=1)
#print('%s notebooks' % df_nb.shape[0])
df_nb.head()

,nb_id,html_url,name
0,0,https://github.com/dalequark/emotivExperiment/...,EmotivDataAnalysis.ipynb
1,1,https://github.com/kevcisme/madelon_redux/blob...,Part_IV_Project_3-checkpoint_BASE_63907.ipynb
2,2,https://github.com/HaraldoFilho/DLND-Projects/...,_.ipynb
3,3,https://github.com/mhjensen/CPMLS/blob/4a5b37e...,csexmas2015.ipynb
4,4,https://github.com/freqn/atom_configuration/bl...,jupyter.ipynb


In [85]:
with zipfile.ZipFile('bb2733859v_2_1.zip', 'r') as nbzip:
    with nbzip.open('nb_1.ipynb') as test:
        print(test.read())
test = archive1.read('nb_1.ipynb')

b''


In [24]:
folder = 'bb2733859v_2_1'
filenames = os.listdir(folder)
print(len(filenames))
def get_ids(path):
    filenames = os.listdir(path)
    ids = []
    for i in range(len(filenames)):
        if filenames[i].startswith('nb_') and filenames[i].endswith('.ipynb'):
            current_id = int(filenames[i][3:-6])
            ids.append(current_id)
        else:
            print('miss')
    return ids

ids = get_ids(folder)
print(len(ids)) # check same length
# row = df_nb.loc[df_nb['nb_id'] == 0]

198778
198778


In [27]:
# extract markdown, language and comments
# path = "./sample_data/data/notebooks/"
# notebook_loc = path + 'nb_1222.ipynb'
def get_text(path):
    markdown = []
    comments = []
    with open(path) as fp:
        try:
            data = nbformat.read(fp, nbformat.NO_CONVERT)
            #data = json.load(fp)
        except:
            return None, None, None
    if 'cells' in data:
        cells = data['cells']
        if 'metadata' in data:
            if 'kernelspec' in data['metadata']:
                if 'language' in data['metadata']['kernelspec']:
                    language = data['metadata']['kernelspec']['language']
                else:
                    language = None
            else:
                language = None
        else:
            language = None
        md_cells = [c for c in cells if c['cell_type'] == 'markdown']
        code_cells = [c for c in cells if c['cell_type'] == 'code']
        for cell in md_cells:
            markdown.append(cell['source'])

        # find comments '# ' for R and Python
        for cell in code_cells:
            source = cell['source']
            string = ''
            if source != None:
                for item in source:
                    string = string + str(item)
                if '# ' in string:
                    comments.append(string)
        return language, markdown, comments
    else:
        return None, None, None

# l,m,c = get_text(notebook_loc)
#print(c)
print(get_text('bb2733859v_2_1/nb_0.ipynb'))

(None, None, None)


In [ ]:
# append all extra columns to dataframe and export to csv
final_df = pd.DataFrame(columns = COLUMNS)
for i in range(len(ids)):
    path = folder + '/nb_' + str(ids[i]) + '.ipynb'
    language, markdown, comments = get_text(path)
    if language != None or markdown != None or comments != None:
        row = df_nb.loc[df_nb['nb_id'] == ids[i]]
        final_df = final_df.append({COLUMNS[0]:row['nb_id'].values[0],
                                    COLUMNS[1]:row['html_url'].values[0],
                                    COLUMNS[2]:row['name'].values[0],
                                    COLUMNS[3]:language,
                                    COLUMNS[4]:markdown,
                                    COLUMNS[5]:comments},
                                   ignore_index=True)
# save the dataframe to csv
final_df.to_csv('df_bb2733859v_2_1.csv')
final_df.head()

O:\Anaconda3\lib\site-packages\nbformat\validator.py:251: UserWarning: No schema for validating v2 notebooks
  warnings.warn("No schema for validating v%s notebooks" % version, UserWarning)
O:\Anaconda3\lib\site-packages\nbformat\validator.py:251: UserWarning: No schema for validating v1 notebooks
  warnings.warn("No schema for validating v%s notebooks" % version, UserWarning)


In [ ]:
es_df = pd.read_csv('df_bb2733859v_2_1.csv')

bulk_data = []

for index, row in es_df.iterrows():
    data_dict = {}
    for i in range(len(row)):
        data_dict[es_df.columns[i]] = row[i]
    op_dict = {
        "index": {
            "_index": INDEX,
            "_type": TYPE,
            "_id": data_dict['nb_id']
        }
    }
    bulk_data.append(op_dict)
    bulk_data.append(data_dict)

In [ ]:
# Initialise the elastic search index
init_index = {
    "settings" : {
        "number_of_shards": 5,
        "number_of_replicas": 1
        "analysis": {
            "analyzer": {
                "text_analyzer": {"type": "standard"}
            }
        }
    },
    'mappings': {
        'notebook': {
            'properties': {
                'nb_id': {'index': 'not_analyzed', 'type': 'string'},
                'html_url': {'index': 'not_analyzed', 'type': 'string'},
                'name': {'index': 'not_analyzed', 'type': 'string'},
                'language': {'index': 'not_analyzed', 'type': 'string'},
                'markdown': {'index': 'analyzed', 'type': 'string'},
                'comments': {'index': 'analyzed', 'type': 'string'},
            }
        }
    }
}
es.indices.create(index = INDEX, body = init_index)

In [ ]:
res = es.bulk(index = INDEX, body = bulk_data)

# check if the data and structure is in elasticsearch
es.search(body={"query": {"match_all": {}}}, index = INDEX)
es_indices.get_mapping(index = INDEX)

# datadict = df_nb.todict()
# res = es.index(index=INDEX, doc_type='notebook', body=datadict)